<a href="https://colab.research.google.com/github/kumaran0012/DImensionality-Reduction-Using-K-Means-for-skeleton-Code/blob/main/dimensionality_reduction_K_means_for_skeletan_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.


<ipython-input-1-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-1-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [4]:
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())


In [6]:
# Solution to Task 2: Encode class labels
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())

In [7]:
# Task 3: Scale the features using StandardScaler
# YOUR CODE HERE: Apply StandardScaler to df

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [8]:
def feature_reduction(df_scaled, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(df_scaled.T)  # Transpose for feature-wise clustering
    return kmeans.labels_

In [9]:
n_clusters = 10  # Number of feature clusters
feature_clusters = feature_reduction(df_scaled, n_clusters)

In [10]:
df_reduced = np.hstack([
    df_scaled[:, feature_clusters == cluster].mean(axis=1, keepdims=True)
    for cluster in np.unique(feature_clusters)
])


In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_reduced, encoded_y, test_size=0.3, random_state=42)


In [12]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scale features again after reduction
    ('classifier', LogisticRegression(max_iter=200, random_state=42))
])


In [13]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 LogisticRegression(max_iter=200, random_state=42))])

In [14]:
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8250
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.85      0.82       366
           1       0.82      0.82      0.82       304
           2       0.81      0.76      0.78       311
           3       0.84      0.63      0.72       386
           4       0.74      0.91      0.82       411
           5       0.95      0.94      0.94       428

    accuracy                           0.83      2206
   macro avg       0.83      0.82      0.82      2206
weighted avg       0.83      0.83      0.82      2206



In [16]:
# TASK 7 - K-Means for dimensionality reduction
n_clusters = 10  # Choose the number of clusters for dimensionality reduction
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Transpose the data to treat features as data points for clustering
kmeans.fit(df_scaled.T)

# Identify the indices of the selected features (one feature per cluster, closest to the cluster center)
selected_features_indices = np.argmin(kmeans.transform(df_scaled.T), axis=0)
selected_features = df.iloc[:, selected_features_indices]


In [18]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8250
